In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from operator import add
class LazySegmentTree:
  def __init__(self, data, f=add, fl=lambda i, x, cnt: i+x*cnt, fll=add, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(data)
    self.L = [default] * self.len + data
    self.lazy = [ldefault] * (self.len + 1)
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.L[i] = self.f(self.L[i*2], self.L[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _push(self, idx, val) :
    self.L[idx] = self.fl(self.L[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.L[idx] = self.f(self.L[idx*2], self.L[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.L[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.L[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol():
  def ett(root) :
    cnt = 0
    IN = [0] * (N+1)
    OUT = [0] * (N+1)
    S = [(root, -1)]
    while S:
      x, p = S[-1]

      if IN[x] == 0:
        cnt += 1
        IN[x] = cnt
        for nx in G[x]:
          if nx == p: continue
          S.append((nx, x))
      else:
        S.pop()
        OUT[x] = cnt
    
    return IN, OUT
  
  N = int(input())
  G = [[] for _ in range(N+1)]
  for i, v in enumerate(map(int, input().split()), 1):
    if v == 0 : continue
    G[v].append(i)

  lst = LazySegmentTree([0] + [1] * N, fl=lambda i, x, cnt: cnt if x == 1 else 0, fll=lambda i, x: x, ldefault=None)
  IN, OUT = ett(1)
  ans = []
  for _ in range(int(input())):
    q, x = map(int, input().split())
    if q == 1 :
      lst.update(IN[x]+1, OUT[x], 1) #i번 직원을 제외한 모든 부하직원
    elif q == 2 :
      lst.update(IN[x]+1, OUT[x], 0)
    elif q == 3 :
      ans.append(lst.query(IN[x]+1, OUT[x]))
    
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 풀이
- `i`번 직원을 상사로 두는 모든 직원 = `i` 번 직원을 제외한 `i`번 직원의 모든 부하 직원
- ett로 나온 IN[x] 에 1을 더하면 자기 자신노드를 제외시킬 수 있다.
- +1만큼 더했기 때문에 lazy공간도 1개만큼 더 늘려줘야 한다.